In [14]:
import pandas as pd
import pyomo.environ as pyo

# Calculating expected generation from renewables

In [15]:
load = pd.read_csv('DataC_hourlyload.csv', sep = ';')

In [16]:
renewables = pd.read_csv('DataC_nondispgeneration.csv',sep = ',')

In [17]:
load

,Date,Start,End,Total (grid load) [MWh] Calculated resolutions
0,"Mar 7, 2023",12:00 AM,1:00 AM,50219.75
1,"Mar 7, 2023",1:00 AM,2:00 AM,48657.00
2,"Mar 7, 2023",2:00 AM,3:00 AM,48689.00
3,"Mar 7, 2023",3:00 AM,4:00 AM,49895.50
4,"Mar 7, 2023",4:00 AM,5:00 AM,51134.75
5,"Mar 7, 2023",5:00 AM,6:00 AM,54662.25
6,"Mar 7, 2023",6:00 AM,7:00 AM,61562.75
7,"Mar 7, 2023",7:00 AM,8:00 AM,66396.00
8,"Mar 7, 2023",8:00 AM,9:00 AM,68569.50
9,"Mar 7, 2023",9:00 AM,10:00 AM,70653.25


In [18]:
load['hour']=load.index
load=load.drop(['Start','End'],axis=1)
load=load[['Date','hour','Total (grid load) [MWh] Calculated resolutions']]
load.columns = ['Date','hour','load']

In [19]:
load

,Date,hour,load
0,"Mar 7, 2023",0,50219.75
1,"Mar 7, 2023",1,48657.00
2,"Mar 7, 2023",2,48689.00
3,"Mar 7, 2023",3,49895.50
4,"Mar 7, 2023",4,51134.75
5,"Mar 7, 2023",5,54662.25
6,"Mar 7, 2023",6,61562.75
7,"Mar 7, 2023",7,66396.00
8,"Mar 7, 2023",8,68569.50
9,"Mar 7, 2023",9,70653.25


In [20]:
renewables

,hour,generation wind,probability wind,generation solar,probability solar
0,0-5,20000,0.3,0,1.0
1,0-5,25000,0.4,0,1.0
2,0-5,30000,0.2,0,1.0
3,0-5,2000,0.1,0,1.0
4,6-10,20000,0.3,1000,0.2
5,6-10,25000,0.4,4000,0.2
6,6-10,30000,0.2,6000,0.3
7,6-10,2000,0.1,9000,0.3
8,11-14,20000,0.3,1000,0.2
9,11-14,25000,0.4,6000,0.2


In [21]:
def create_renewable_scenario(ren):
    hours = [n for n in range(0,24)]
    generation = [0]*24
    generation_df = pd.DataFrame({'hour': hours, 'generation':generation})
    expected_values = calculate_expected_values(ren)
    types = ['wind','solar']
    for n in range(0,2):
        generation_df.loc[0:5,'generation_'+ types[n] ] = expected_values[n][0]
        generation_df.loc[6:10,'generation_'+ types[n] ] = expected_values[n][1]
        generation_df.loc[11:14,'generation_'+ types[n] ] = expected_values[n][2]
        generation_df.loc[15:19,'generation_'+ types[n] ] = expected_values[n][3]
        generation_df.loc[20:23,'generation_'+ types[n] ] = expected_values[n][4]
    return generation_df

In [22]:
def calculate_expected_values(ren):
    expected_value_solar_list = []
    expected_value_wind_list = []
    for n in range(0,5):
        expected_value_wind = sum(ren.loc[n*4:n*4+3,'generation wind']*ren.loc[n*4:n*4+3,'probability wind'])
        expected_value_solar = sum(ren.loc[n*4:n*4+3,'generation solar']*ren.loc[n*4:n*4+3,'probability solar'])
        expected_value_solar_list.append(expected_value_solar)
        expected_value_wind_list.append(expected_value_wind)
    return expected_value_wind_list,expected_value_solar_list  

In [23]:
renewable_expected = create_renewable_scenario(renewables)
load = pd.merge(load,renewable_expected)

In [24]:
renewable_expected

,hour,generation,generation_wind,generation_solar
0,0,0,22200.0,0.0
1,1,0,22200.0,0.0
2,2,0,22200.0,0.0
3,3,0,22200.0,0.0
4,4,0,22200.0,0.0
5,5,0,22200.0,0.0
6,6,0,22200.0,5500.0
7,7,0,22200.0,5500.0
8,8,0,22200.0,5500.0
9,9,0,22200.0,5500.0


In [61]:
wind = renewable_expected['generation_wind'].values.tolist()
print(wind)

[22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 22200.0, 24700.0, 24700.0, 24700.0, 24700.0]


In [63]:
solar = renewable_expected['generation_solar'].values.tolist()
print(solar)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5500.0, 5500.0, 5500.0, 5500.0, 5500.0, 10400.0, 10400.0, 10400.0, 10400.0, 5500.0, 5500.0, 5500.0, 5500.0, 5500.0, 0.0, 0.0, 0.0, 0.0]


In [25]:
renewable_expected

,hour,generation,generation_wind,generation_solar
0,0,0,22200.0,0.0
1,1,0,22200.0,0.0
2,2,0,22200.0,0.0
3,3,0,22200.0,0.0
4,4,0,22200.0,0.0
5,5,0,22200.0,0.0
6,6,0,22200.0,5500.0
7,7,0,22200.0,5500.0
8,8,0,22200.0,5500.0
9,9,0,22200.0,5500.0


In [26]:
gen_params = pd.read_csv('DataC_generatorparams.csv',sep = ';')

In [27]:
load['load']

0     50219.75
1     48657.00
2     48689.00
3     49895.50
4     51134.75
5     54662.25
6     61562.75
7     66396.00
8     68569.50
9     70653.25
10    72214.50
11    72710.50
12    71735.50
13    70005.50
14    68428.00
15    66736.25
16    65774.25
17    65269.50
18    66352.75
19    66179.75
20    62462.25
21    59083.00
22    56607.00
23    53307.25
Name: load, dtype: float64

In [28]:
gen_params

,generator,a1,a2,pmin,pmax
0,1,5,0.100,2000,16000
1,2,6,0.200,1800,23000
2,3,7,0.015,100,12000
3,4,4,0.018,500,15000


In [29]:
M = 100000000

# Creating model

In [30]:
model = pyo.ConcreteModel()

In [31]:
model.a1 = gen_params["a1"]
model.a2 = gen_params["a2"]
model.pmin = gen_params["pmin"]
model.pmax = gen_params["pmax"]
model.load = load['load']
model.expected_wind = load['generation_wind']
model.expected_solar = load['generation_solar']


In [32]:
def objective_function(model):
    obj = 0
    for n in model.n:
        # generator costs
        generator_cost_1 = model.a1[0]*model.p1[n]+model.a2[0]*model.p1[n]**2
        generator_cost_2 = model.a1[1]*model.p2[n]+model.a2[1]*model.p2[n]**2
        generator_cost_3 = model.a1[2]*model.p3[n]+model.a2[2]*model.p3[n]**2
        generator_cost_4 = model.a1[3]*model.p4[n]+model.a2[3]*model.p4[n]**2
        generator_costs = generator_cost_1 + generator_cost_2 + generator_cost_3 + generator_cost_4
        # imbalance costs
        imbalance_cost = model.imbalance[n]*1000
        # min/max valiation costs
        min_costs = model.p1_pen_min[n] + model.p2_pen_min[n] + model.p3_pen_min[n] + model.p4_pen_min[n]
        max_costs = model.p1_pen_max[n] + model.p2_pen_max[n] + model.p3_pen_max[n] + model.p4_pen_max[n]
#         obj += generator_costs +  min_costs + max_costs
        obj += generator_costs + imbalance_cost + min_costs + max_costs
    return obj

## Define the dimension

In [33]:
model.n = pyo.Set(initialize=range(0,24))

In [34]:
model.p1 = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p2 = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p3 = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p4 = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.u1 = pyo.Var(model.n, domain = pyo.Binary)
model.u2 = pyo.Var(model.n, domain = pyo.Binary)
model.u3 = pyo.Var(model.n, domain = pyo.Binary)
model.u4 = pyo.Var(model.n, domain = pyo.Binary)
model.p1_pen_max = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p2_pen_max = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p3_pen_max = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p1_pen_min = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p2_pen_min = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p3_pen_min = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p4_pen_max = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.p4_pen_min = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.c1_min = pyo.Var(model.n, domain = pyo.Binary)
model.c1_max = pyo.Var(model.n, domain = pyo.Binary)
model.c2_min = pyo.Var(model.n, domain = pyo.Binary)
model.c2_max = pyo.Var(model.n, domain = pyo.Binary)
model.c3_min = pyo.Var(model.n, domain = pyo.Binary)
model.c3_max = pyo.Var(model.n, domain = pyo.Binary)
model.c4_min = pyo.Var(model.n, domain = pyo.Binary)
model.c4_max = pyo.Var(model.n, domain = pyo.Binary)
model.imbalance = pyo.Var(model.n, domain = pyo.Reals)
model.uwind = pyo.Var(model.n, domain = pyo.Binary)
model.usolar = pyo.Var(model.n, domain = pyo.Binary)
model.wind_prod = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.solar_prod = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.imbalance_c = pyo.Var(model.n, domain = pyo.Binary)
model.pos_imbalance = pyo.Var(model.n, domain = pyo.NonNegativeReals)
model.neg_imbalance = pyo.Var(model.n, domain = pyo.NonNegativeReals)

In [35]:
# # gen__min 
# def constraint_1_min_fun_1(model, index):
#      return model.p1_pen_min[index] <=M*model.u1[index]
# def constraint_1_min_fun_2(model, index):
#      return model.p1_pen_min[index] >=0
# def constraint_1_min_fun_3(model, index):
#      return model.p1_pen_min[index] <= M*(1-model.u1[index])
# def constraint_1_min_fun_4(model, index):
#      return model.p1_pen_min[index] >= gen_params.loc[0,'pmin'] - model.p1[index]
# def constraint_1_min_fun_5(model, index):
#      return model.p1_pen_min[index] <= gen_params.loc[0,'pmin'] - model.p1[index] + M*(1-model.u1[index]) 
    
# # gen__max
# def constraint_1_max_fun_1(model, index):
#      return model.p1_pen_max[index] <=M*model.u1[index]
# def constraint_1_max_fun_2(model, index):
#      return model.p1_pen_max[index] >=0
# def constraint_1_max_fun_3(model, index):
#      return model.p1_pen_max[index] >= gen_params.loc[0,'pmax'] - model.p1[index] 
# def constraint_1_max_fun_4(model, index):
#      return model.p1_pen_max[index] <= gen_params.loc[0,'pmax'] - model.p1[index] + M*(1-model.u1[index])
# def constraint_1_max_fun_5(model, index):
#     return model.p1_pen_max[index] <= gen_params.loc[0,'pmax'] - model.p1[index] 

In [36]:
#gen_1_min (big M formulation of max(0,model.p1_pen_min) so that we don't get negative panality if we are below min)
def constraint_1_min_fun_1(model, index):
    return model.p1_pen_min[index] >=0 
def constraint_1_min_fun_2(model, index):
    return model.p1_pen_min[index] >= (gen_params.loc[0,'pmin'] - model.p1[index])*model.u1[index] 
def constraint_1_min_fun_3(model, index):
    return model.p1_pen_min[index] <= M * model.c1_min[index]
def constraint_1_min_fun_4(model, index):
    return model.p1_pen_min[index] <= gen_params.loc[0,'pmin'] - model.p1[index]  + M*(1-model.c1_min[index])
# gen_1_max
def constraint_1_max_fun_1(model, index):
    return model.p1_pen_max[index] >= 0
def constraint_1_max_fun_2(model, index):
    return model.p1_pen_max[index] >= model.p1[index] - gen_params.loc[0,'pmax']
def constraint_1_max_fun_3(model, index):
    return model.p1_pen_max[index] <= M * model.c1_max[index]
def constraint_1_max_fun_4(model, index):
    return model.p1_pen_max[index] <= model.p1[index] - gen_params.loc[0,'pmax'] + M*(1-model.c1_max[index])
#gen_2_min
def constraint_2_min_fun_1(model, index):
    return model.p2_pen_min[index] >=0
def constraint_2_min_fun_2(model, index):
    return model.p2_pen_min[index] >= (gen_params.loc[1,'pmin'] - model.p2[index])*model.u2[index]
def constraint_2_min_fun_3(model, index):
    return model.p2_pen_min[index] <= M * model.c2_min[index]
def constraint_2_min_fun_4(model, index):
    return model.p2_pen_min[index] <= gen_params.loc[1,'pmin'] - model.p2[index] + M*(1-model.c2_min[index])
# gen_2_max
def constraint_2_max_fun_1(model, index):
    return model.p2_pen_max[index] >=0
def constraint_2_max_fun_2(model, index):
    return model.p2_pen_max[index] >= model.p2[index] - gen_params.loc[1,'pmax']
def constraint_2_max_fun_3(model, index):
    return model.p2_pen_max[index] <= M * model.c2_max[index]
def constraint_2_max_fun_4(model, index):
    return model.p2_pen_max[index] <= model.p2[index] - gen_params.loc[1,'pmax'] + M*(1-model.c2_max[index])
# gen_3_min
def constraint_3_min_fun_1(model, index):
    return model.p3_pen_min[index] >= 0
def constraint_3_min_fun_2(model, index):
    return model.p3_pen_min[index] >= (gen_params.loc[2,'pmin'] - model.p3[index])*model.u3[index]
def constraint_3_min_fun_3(model, index):
    return model.p3_pen_min[index] <= M * model.c3_min[index]
def constraint_3_min_fun_4(model, index):
    return model.p3_pen_min[index] <= gen_params.loc[2,'pmin'] - model.p3[index] + M*(1-model.c3_min[index])
# gen_3_max
def constraint_3_max_fun_1(model, index):
    return model.p3_pen_max[index] >=0
def constraint_3_max_fun_2(model, index):
    return model.p3_pen_max[index] >= model.p3[index] - gen_params.loc[2,'pmax']
def constraint_3_max_fun_3(model, index):
    return model.p3_pen_max[index] <= M * model.c3_max[index]
def constraint_3_max_fun_4(model, index):
    return model.p3_pen_max[index] <= model.p3[index] - gen_params.loc[2,'pmax'] + M*(1-model.c3_max[index])

# gen_4_min
def constraint_4_min_fun_1(model, index):
    return model.p4_pen_min[index] >= 0
def constraint_4_min_fun_2(model, index):
    return model.p4_pen_min[index] >= (gen_params.loc[3,'pmin'] - model.p4[index])*model.u4[index]
def constraint_4_min_fun_3(model, index):
    return model.p4_pen_min[index] <= M * model.c4_min[index]
def constraint_4_min_fun_4(model, index):
    return model.p4_pen_min[index] <= gen_params.loc[3,'pmin'] - model.p4[index] + M*(1-model.c4_min[index])
# gen_4_max
def constraint_4_max_fun_1(model, index):
    return model.p4_pen_max[index] >=0
def constraint_4_max_fun_2(model, index):
    return model.p4_pen_max[index] >= model.p4[index] - gen_params.loc[3,'pmax']
def constraint_4_max_fun_3(model, index):
    return model.p4_pen_max[index] <= M * model.c4_max[index]
def constraint_4_max_fun_4(model, index):
    return model.p4_pen_max[index] <= model.p4[index] - gen_params.loc[3,'pmax'] + M*(1-model.c4_max[index])

# constraints for u = 0 when power = 0
def constraint_u_rule_1(model, index):
    return model.p1[index] <= M * model.u1[index]
def constraint_u_rule_2(model, index):
    return model.p2[index] <= M * model.u2[index]
def constraint_u_rule_3(model, index):
    return model.p3[index] <= M * model.u3[index]
def constraint_u_rule_4(model, index):
    return model.p4[index] <= M * model.u4[index]

# constraints for wind and solar
def wind_constraint(model,index):
    return model.wind_prod[index] == model.expected_wind[index]*model.uwind[index]
def solar_constraint(model, index):
    return model.solar_prod[index] == model.expected_solar[index]*model.usolar[index]

In [37]:
def constraint_imbalance_1(model,index):
    imbalance = model.p1[index] + model.p2[index] + model.p3[index] + model.p4[index]- model.load[index] + model.wind_prod[index] + model.solar_prod[index]
    return imbalance == model.pos_imbalance[index] - model.neg_imbalance[index]   
def constraint_imbalance_2(model,index):
    return model.imbalance[index] == model.pos_imbalance[index] + model.neg_imbalance[index]
def constraint_imbalance_3(model,index):
    return model.pos_imbalance[index] <= M*model.imbalance_c[index]
def constraint_imbalance_4(model,index):
    return model.neg_imbalance[index] <= M*(1-model.imbalance_c[index])

In [38]:
model.p1_min_1 = pyo.Constraint(model.n,rule = constraint_1_min_fun_1 )
model.p1_min_2 = pyo.Constraint(model.n,rule = constraint_1_min_fun_2 )
model.p1_min_3 = pyo.Constraint(model.n,rule = constraint_1_min_fun_3 )
model.p1_min_4 = pyo.Constraint(model.n,rule = constraint_1_min_fun_4 )
# model.p1_min_5 = pyo.Constraint(model.n,rule = constraint_1_min_fun_4 )

model.p2_min_1 = pyo.Constraint(model.n,rule = constraint_2_min_fun_1 )
model.p2_min_2 = pyo.Constraint(model.n,rule = constraint_2_min_fun_2 )
model.p2_min_3 = pyo.Constraint(model.n,rule = constraint_2_min_fun_3 )
model.p2_min_4 = pyo.Constraint(model.n,rule = constraint_2_min_fun_4 )

model.p3_min_1 = pyo.Constraint(model.n,rule = constraint_3_min_fun_1 )
model.p3_min_2 = pyo.Constraint(model.n,rule = constraint_3_min_fun_2 )
model.p3_min_3 = pyo.Constraint(model.n,rule = constraint_3_min_fun_3 )
model.p3_min_4 = pyo.Constraint(model.n,rule = constraint_3_min_fun_4 )

model.p4_min_1 = pyo.Constraint(model.n,rule = constraint_4_min_fun_1 )
model.p4_min_2 = pyo.Constraint(model.n,rule = constraint_4_min_fun_2 )
model.p4_min_3 = pyo.Constraint(model.n,rule = constraint_4_min_fun_3 )
model.p4_min_4 = pyo.Constraint(model.n,rule = constraint_4_min_fun_4 )

model.p1_max_1 = pyo.Constraint(model.n,rule = constraint_1_max_fun_1 )
model.p1_max_2 = pyo.Constraint(model.n,rule = constraint_1_max_fun_2 )
model.p1_max_3 = pyo.Constraint(model.n,rule = constraint_1_max_fun_3 )
model.p1_max_4 = pyo.Constraint(model.n,rule = constraint_1_max_fun_4 )
# model.p1_max_5 = pyo.Constraint(model.n,rule = constraint_1_max_fun_5 )

model.p2_max_1 = pyo.Constraint(model.n,rule = constraint_2_max_fun_1 )
model.p2_max_2 = pyo.Constraint(model.n,rule = constraint_2_max_fun_2 )
model.p2_max_3 = pyo.Constraint(model.n,rule = constraint_2_max_fun_3 )
model.p2_max_4 = pyo.Constraint(model.n,rule = constraint_2_max_fun_4 )

model.p3_max_1 = pyo.Constraint(model.n,rule = constraint_3_max_fun_1 )
model.p3_max_2 = pyo.Constraint(model.n,rule = constraint_3_max_fun_2 )
model.p3_max_3 = pyo.Constraint(model.n,rule = constraint_3_max_fun_3 )
model.p3_max_4 = pyo.Constraint(model.n,rule = constraint_3_max_fun_4 )

model.p4_max_1 = pyo.Constraint(model.n,rule = constraint_4_max_fun_1 )
model.p4_max_2 = pyo.Constraint(model.n,rule = constraint_4_max_fun_2 )
model.p4_max_3 = pyo.Constraint(model.n,rule = constraint_4_max_fun_3 )
model.p4_max_4 = pyo.Constraint(model.n,rule = constraint_4_max_fun_4 )

model.u1_rule = pyo.Constraint(model.n,rule = constraint_u_rule_1 )
model.u2_rule = pyo.Constraint(model.n,rule = constraint_u_rule_2 )
model.u3_rule = pyo.Constraint(model.n,rule = constraint_u_rule_3 )
model.u4_rule = pyo.Constraint(model.n,rule = constraint_u_rule_4 )
#model.hard_imbalance = pyo.Constraint(model.n,rule = constraint_hard_imbalance)

model.imbalance_1 = pyo.Constraint(model.n,rule = constraint_imbalance_1)
model.imbalance_2 = pyo.Constraint(model.n,rule = constraint_imbalance_2)
model.imbalance_3 = pyo.Constraint(model.n,rule = constraint_imbalance_3)
model.imbalance_4 = pyo.Constraint(model.n,rule = constraint_imbalance_4)
# model.imbalance_3 = pyo.Constraint(model.n,rule = constraint_imbalance_1)
# model.imbalance_4 = pyo.Constraint(model.n,rule = constraint_imbalance_2)
# model.imbalance_5 = pyo.Constraint(model.n,rule = constraint_imbalance_2)
model.wind_cons = pyo.Constraint(model.n,rule = wind_constraint)
model.solar_cons = pyo.Constraint(model.n,rule = solar_constraint)
# model.solar_cons_1 = pyo.Constraint(model.n,rule = solar_constraint_1)
# model.solar_cons_2 = pyo.Constraint(model.n,rule = solar_constraint_2)


In [39]:
model.obj = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

In [40]:
optim = pyo.SolverFactory('gurobi')

In [41]:
result = optim.solve(model)

In [42]:
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 227989944.69313443
  Upper bound: 227990281.26598576
  Number of objectives: 1
  Number of constraints: 1008
  Number of variables: 758
  Number of binary variables: 350
  Number of integer variables: 350
  Number of continuous variables: 408
  Number of nonzeros: 1982
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Mo

In [48]:
u1_values = [float(val) if val else 0 for val in model.u1.get_values().values()]
u2_values = [float(val) if val else 0 for val in model.u2.get_values().values()]
u3_values = [float(val) if val else 0 for val in model.u3.get_values().values()]
u4_values = [float(val) if val else 0 for val in model.u4.get_values().values()]
p1_values = [float(val) if val else 0 for val in model.p1.get_values().values()]
p2_values = [float(val) if val else 0 for val in model.p2.get_values().values()]
p3_values = [float(val) if val else 0 for val in model.p3.get_values().values()]
p4_values = [float(val) if val else 0 for val in model.p4.get_values().values()]
uwind_values = [float(val) if val else 0 for val in model.uwind.get_values().values()]
usolar_values = [float(val) if val else 0 for val in model.usolar.get_values().values()]

print("u1:", u1_values)
print("u2:", u2_values)
print("u3:", u3_values)
print("p1:", p1_values)
print("p2:", p2_values)
print("p3:", p3_values)
print("p4:", p4_values)
print("uwind:", uwind_values)
print("usolar:", usolar_values)


u1: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
u2: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
u3: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
p1: [2047.1882591093117, 1937.9392712550612, 1940.2712550607287, 2023.5587044534414, 2113.8684210526317, 2370.9331983805664, 2472.9939271255057, 2827.238866396761, 2985.6315789473683, 3137.483805668016, 3251.2591093117408, 2928.83469840802, 2859.2672064777325, 2733.1943319838056, 2616.2105263157887, 2852.0344129554655, 2781.9291497975705, 2745.145748987854, 2824.0870445344126, 2811.47975708502, 2759.1923076923076, 2510.9068825910927, 2330.4696356275304, 2090.002024291498]
p2: [1023.5941295546559, 966.4696356275306, 967.6356275303643, 1011.7793522267207, 1056.9342105263158, 1185.4665991902832, 1236.4969635627529, 1413.

In [58]:
#model..display()

usolar : Size=24, Index=n
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :     1 : False :  True : Binary
      1 :     0 :  None :     1 : False :  True : Binary
      2 :     0 :  None :     1 : False :  True : Binary
      3 :     0 :  None :     1 : False :  True : Binary
      4 :     0 :  None :     1 : False :  True : Binary
      5 :     0 :  None :     1 : False :  True : Binary
      6 :     0 :   1.0 :     1 : False : False : Binary
      7 :     0 :   1.0 :     1 : False : False : Binary
      8 :     0 :   1.0 :     1 : False : False : Binary
      9 :     0 :   1.0 :     1 : False : False : Binary
     10 :     0 :   1.0 :     1 : False : False : Binary
     11 :     0 :   1.0 :     1 : False : False : Binary
     12 :     0 :   1.0 :     1 : False : False : Binary
     13 :     0 :   1.0 :     1 : False : False : Binary
     14 :     0 :   1.0 :     1 : False : False : Binary
     15 :     0 :   1.0 :     1 : False : False : Binary
     

In [49]:
import json

data = {
    'name': 'Group_F',
    'task': 'C',
    'u1': [1 if val > 0 else 0 for val in u1_values],
    'u2': [1 if val > 0 else 0 for val in u2_values],
    'u3': [1 if val > 0 else 0 for val in u3_values],
    'u4': [1 if val > 0 else 0 for val in u4_values],
    'p1': p1_values,
    'p2': p2_values,
    'p3': p3_values,
    'p4': p4_values,
    'uwind': [1 if val > 0 else 0 for val in uwind_values],
    'usolar': [1 if val > 0 else 0 for val in usolar_values]
}

# Save data to JSON file
with open('results_F1.json', 'w') as json_file:
    json.dump(data, json_file)


In [47]:
model.u1.display()

u1 : Size=24, Index=n
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :   1.0 :     1 : False : False : Binary
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :   1.0 :     1 : False : False : Binary
      4 :     0 :   1.0 :     1 : False : False : Binary
      5 :     0 :   1.0 :     1 : False : False : Binary
      6 :     0 :   1.0 :     1 : False : False : Binary
      7 :     0 :   1.0 :     1 : False : False : Binary
      8 :     0 :   1.0 :     1 : False : False : Binary
      9 :     0 :   1.0 :     1 : False : False : Binary
     10 :     0 :   1.0 :     1 : False : False : Binary
     11 :     0 :   1.0 :     1 : False : False : Binary
     12 :     0 :   1.0 :     1 : False : False : Binary
     13 :     0 :   1.0 :     1 : False : False : Binary
     14 :     0 :   1.0 :     1 : False : False : Binary
     15 :     0 :   1.0 :     1 : False : False : Binary
     16 :

In [ ]:
model.pprint()